# 3-Bit Flip-Flop Working Memory Task

This notebook demonstrates how to apply the RNN framework to a different task: the **3-bit flip-flop** working memory task.

## Task Description

The flip-flop task tests a network's ability to maintain state over time:
- **3 input channels**: Binary pulses that command state toggles
- **3 output channels**: Current state of each flip-flop (0 or 1)
- **Rule**: An input pulse on channel *i* toggles flip-flop *i*
- **Challenge**: Remember and maintain state with no continuous input

This is a classic working memory benchmark from:
- Sussillo & Barak (2013) - *Opening the Black Box*
- Mante et al. (2013) - *Context-dependent computation*

## What We'll Cover

1. ✅ Load flip-flop dataset from `src.data.flipflop`
2. 🧠 Train **CT-RNN** on working memory task
3. ⚖️ Train **Balanced Rate Network** for comparison
4. 📊 **PCA visualization** of hidden state dynamics
5. 🎯 **Fixed point analysis** of learned representations
6. 📈 Training curves and test accuracy
7. 🔍 Compare architectures on working memory

---
## 1. Setup

In [ ]:
# Google Colab setup
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab - installing dependencies...")
    !pip install -q torch torchdiffeq matplotlib scipy tqdm
    !git clone -q https://github.com/CNNC-Lab/RNNs-tutorial.git
    %cd RNNs-tutorial
    print("✓ Setup complete!")

from src import setup_environment, check_dependencies

check_dependencies()
device = setup_environment()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.decomposition import PCA

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

---
## 2. Load Flip-Flop Dataset

In [ ]:
from src.data.flipflop import create_flipflop_dataloaders, plot_flipflop_trial

# Create dataloaders
train_loader, val_loader, test_loader, info = create_flipflop_dataloaders(
    train_trials=800,
    val_trials=100,
    test_trials=100,
    n_bits=3,
    seq_length=100,
    pulse_prob=0.1,
    batch_size=32,
    seed=42
)

print("Dataset Info:")
for key, value in info.items():
    print(f"  {key}: {value}")

### Visualize Example Trials

In [ ]:
# Get a batch of data
sample_inputs, sample_targets = next(iter(train_loader))

print(f"Batch shapes:")
print(f"  Inputs: {sample_inputs.shape}  # (batch, seq_length, n_bits)")
print(f"  Targets: {sample_targets.shape}")

# Plot first trial
fig = plot_flipflop_trial(
    sample_inputs.numpy(),
    sample_targets.numpy(),
    trial_idx=0,
    figsize=(14, 8)
)
plt.show()

print("\n📌 Notice: Gray pulses toggle the state. Network must remember toggles with no continuous input!")

---
## 3. Train Continuous-Time RNN

In [ ]:
from src.models import ContinuousTimeRNN

# Create CT-RNN
ctrnn = ContinuousTimeRNN(
    input_size=3,
    hidden_size=64,
    output_size=3,
    tau=5.0,  # Longer time constant for working memory
    activation='tanh',
    solver='euler'
).to(device)

n_params = sum(p.numel() for p in ctrnn.parameters())
print(f"CT-RNN Parameters: {n_params:,}")
print(f"Time constant τ: {ctrnn.cell.tau}")

# Test forward pass
with torch.no_grad():
    test_out = ctrnn(sample_inputs[:2].to(device))
    print(f"\nForward pass: {sample_inputs[:2].shape} → {test_out.shape}")

In [ ]:
from src.utils import train_model

print("Training CT-RNN on flip-flop task...\n")

history_ctrnn = train_model(
    model=ctrnn,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=150,
    lr=1e-3,
    weight_decay=1e-5,
    patience=25,
    device=device,
    verbose=True
)

# Save checkpoint
torch.save(ctrnn.state_dict(), 'checkpoints/ctrnn_flipflop_best.pt')
print(f"\n✓ CT-RNN trained! Best val loss: {min(history_ctrnn['val_loss']):.6f}")

---
## 4. Train Balanced Rate Network

In [ ]:
from src.models import BalancedRateNetwork

# Create Balanced Rate Network
balanced_rnn = BalancedRateNetwork(
    input_size=3,
    n_excitatory=96,
    n_inhibitory=24,
    output_size=3,
    tau_e=5.0,  # Match CT-RNN time constant
    tau_i=2.5,
    activation='relu'
).to(device)

n_params_balanced = sum(p.numel() for p in balanced_rnn.parameters())
print(f"Balanced RNN Parameters: {n_params_balanced:,}")
print(f"Architecture: {balanced_rnn.n_excitatory}E + {balanced_rnn.n_inhibitory}I neurons")

# Test forward pass
with torch.no_grad():
    test_out_bal = balanced_rnn(sample_inputs[:2].to(device))
    print(f"\nForward pass: {sample_inputs[:2].shape} → {test_out_bal.shape}")

In [ ]:
print("Training Balanced Rate Network on flip-flop task...\n")

history_balanced = train_model(
    model=balanced_rnn,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=150,
    lr=5e-4,
    weight_decay=1e-5,
    patience=25,
    device=device,
    verbose=True
)

# Save checkpoint
torch.save(balanced_rnn.state_dict(), 'checkpoints/balanced_flipflop_best.pt')
print(f"\n✓ Balanced RNN trained! Best val loss: {min(history_balanced['val_loss']):.6f}")

---
## 5. Training Curves Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# CT-RNN
axes[0].plot(history_ctrnn['train_loss'], label='Train Loss', linewidth=2, alpha=0.8)
axes[0].plot(history_ctrnn['val_loss'], label='Val Loss', linewidth=2, alpha=0.8)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('CT-RNN Training', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# Balanced RNN
axes[1].plot(history_balanced['train_loss'], label='Train Loss', linewidth=2, alpha=0.8, color='orange')
axes[1].plot(history_balanced['val_loss'], label='Val Loss', linewidth=2, alpha=0.8, color='red')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss (MSE)', fontsize=12)
axes[1].set_title('Balanced Rate Network Training', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

---
## 6. Evaluate on Test Set

In [ ]:
from src.utils import evaluate
from src.data.flipflop import compute_flipflop_accuracy

criterion = nn.MSELoss()

# Evaluate CT-RNN
ctrnn.eval()
test_loss_ctrnn, preds_ctrnn, targets_ctrnn = evaluate(ctrnn, test_loader, criterion, device)
accuracy_ctrnn = compute_flipflop_accuracy(preds_ctrnn, targets_ctrnn)

# Evaluate Balanced RNN
balanced_rnn.eval()
test_loss_balanced, preds_balanced, targets_balanced = evaluate(balanced_rnn, test_loader, criterion, device)
accuracy_balanced = compute_flipflop_accuracy(preds_balanced, targets_balanced)

print("\n" + "="*60)
print("TEST SET PERFORMANCE")
print("="*60)
print(f"\nCT-RNN:")
print(f"  MSE Loss: {test_loss_ctrnn:.6f}")
print(f"  Bit Accuracy: {accuracy_ctrnn*100:.2f}%")
print(f"\nBalanced Rate Network:")
print(f"  MSE Loss: {test_loss_balanced:.6f}")
print(f"  Bit Accuracy: {accuracy_balanced*100:.2f}%")
print("\n" + "="*60)

### Visualize Predictions

In [ ]:
# Get test batch for visualization
test_inputs, test_targets = next(iter(test_loader))

# Get predictions
with torch.no_grad():
    test_preds_ctrnn = ctrnn(test_inputs.to(device)).cpu().numpy()
    test_preds_balanced = balanced_rnn(test_inputs.to(device)).cpu().numpy()

test_inputs_np = test_inputs.numpy()
test_targets_np = test_targets.numpy()

In [ ]:
# Plot CT-RNN predictions
fig = plot_flipflop_trial(
    test_inputs_np,
    test_targets_np,
    test_preds_ctrnn,
    trial_idx=0,
    figsize=(14, 8)
)
fig.suptitle('CT-RNN Predictions on Test Trial', fontsize=14, fontweight='bold', y=0.998)
plt.show()

In [ ]:
# Plot Balanced RNN predictions
fig = plot_flipflop_trial(
    test_inputs_np,
    test_targets_np,
    test_preds_balanced,
    trial_idx=0,
    figsize=(14, 8)
)
fig.suptitle('Balanced Rate Network Predictions on Test Trial', fontsize=14, fontweight='bold', y=0.998)
plt.show()

---
## 7. State-Space Visualization with PCA

We'll visualize the hidden state dynamics using PCA to see how the networks represent the 8 possible flip-flop states (2³ = 8 combinations).

In [ ]:
def extract_hidden_states(model, dataloader, device, is_balanced=False):
    """Extract hidden states from model."""
    hidden_states = []
    targets_list = []
    
    model.eval()
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            
            if is_balanced:
                # For balanced network, get hidden states from forward pass
                batch_size, seq_len, _ = inputs.shape
                h = torch.zeros(batch_size, model.n_excitatory + model.n_inhibitory, device=device)
                
                for t in range(seq_len):
                    h = model.cell(inputs[:, t, :], h)
                    hidden_states.append(h.cpu().numpy())
                    targets_list.append(targets[:, t, :].numpy())
            else:
                # For CT-RNN, need to extract hidden states during forward pass
                batch_size, seq_len, _ = inputs.shape
                h = torch.zeros(batch_size, model.hidden_size, device=device)
                
                for t in range(seq_len):
                    # Integrate one step
                    from src.models.ctrnn import CTRNNODEFunc
                    ode_func = CTRNNODEFunc(model.cell, inputs[:, t, :])
                    t_span = torch.tensor([0.0, 1.0], device=device)
                    from torchdiffeq import odeint
                    h_traj = odeint(ode_func, h, t_span, method='euler')
                    h = h_traj[-1]
                    hidden_states.append(h.cpu().numpy())
                    targets_list.append(targets[:, t, :].numpy())
    
    hidden_states = np.vstack(hidden_states)
    targets_array = np.vstack(targets_list)
    
    return hidden_states, targets_array

# Extract hidden states
print("Extracting hidden states from CT-RNN...")
hidden_ctrnn, targets_for_pca = extract_hidden_states(ctrnn, test_loader, device, is_balanced=False)

print("Extracting hidden states from Balanced RNN...")
hidden_balanced, _ = extract_hidden_states(balanced_rnn, test_loader, device, is_balanced=True)

print(f"\nExtracted hidden states:")
print(f"  CT-RNN: {hidden_ctrnn.shape}")
print(f"  Balanced RNN: {hidden_balanced.shape}")
print(f"  Targets: {targets_for_pca.shape}")

In [ ]:
# Perform PCA
pca_ctrnn = PCA(n_components=3)
pca_balanced = PCA(n_components=3)

hidden_ctrnn_pca = pca_ctrnn.fit_transform(hidden_ctrnn)
hidden_balanced_pca = pca_balanced.fit_transform(hidden_balanced)

print(f"PCA explained variance:")
print(f"  CT-RNN: {pca_ctrnn.explained_variance_ratio_[:3]}")
print(f"  Sum: {pca_ctrnn.explained_variance_ratio_[:3].sum():.3f}")
print(f"\n  Balanced RNN: {pca_balanced.explained_variance_ratio_[:3]}")
print(f"  Sum: {pca_balanced.explained_variance_ratio_[:3].sum():.3f}")

In [ ]:
# Create state labels (convert binary targets to integer labels)
state_labels = (targets_for_pca[:, 0] * 4 + targets_for_pca[:, 1] * 2 + targets_for_pca[:, 2]).astype(int)

# Plot PCA
fig = plt.figure(figsize=(16, 7))

# CT-RNN
ax1 = fig.add_subplot(121, projection='3d')
scatter1 = ax1.scatter(
    hidden_ctrnn_pca[:5000, 0],
    hidden_ctrnn_pca[:5000, 1],
    hidden_ctrnn_pca[:5000, 2],
    c=state_labels[:5000],
    cmap='tab10',
    s=5,
    alpha=0.6
)
ax1.set_xlabel('PC1', fontsize=11)
ax1.set_ylabel('PC2', fontsize=11)
ax1.set_zlabel('PC3', fontsize=11)
ax1.set_title('CT-RNN State Space (PCA)', fontsize=13, fontweight='bold')
cbar1 = plt.colorbar(scatter1, ax=ax1, pad=0.1, shrink=0.6)
cbar1.set_label('Flip-Flop State', fontsize=10)

# Balanced RNN
ax2 = fig.add_subplot(122, projection='3d')
scatter2 = ax2.scatter(
    hidden_balanced_pca[:5000, 0],
    hidden_balanced_pca[:5000, 1],
    hidden_balanced_pca[:5000, 2],
    c=state_labels[:5000],
    cmap='tab10',
    s=5,
    alpha=0.6
)
ax2.set_xlabel('PC1', fontsize=11)
ax2.set_ylabel('PC2', fontsize=11)
ax2.set_zlabel('PC3', fontsize=11)
ax2.set_title('Balanced RNN State Space (PCA)', fontsize=13, fontweight='bold')
cbar2 = plt.colorbar(scatter2, ax=ax2, pad=0.1, shrink=0.6)
cbar2.set_label('Flip-Flop State', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📌 Each color represents one of the 8 possible flip-flop states (000 to 111).")
print("   Notice how the network clusters states in the hidden space!")

---
## 8. Fixed Point Analysis

The flip-flop task should have **8 stable fixed points** (one for each state: 000, 001, 010, ..., 111).

In [ ]:
from src.analysis import find_fixed_points, analyze_fixed_point_stability, create_dynamics_fn_from_ctrnn

# CT-RNN fixed points
print("Finding fixed points in CT-RNN...\n")
dynamics_fn_ctrnn = create_dynamics_fn_from_ctrnn(ctrnn, x=torch.zeros(1, 3, device=device))

fixed_points_ctrnn, residuals_ctrnn = find_fixed_points(
    dynamics_fn_ctrnn,
    hidden_size=64,
    n_initial=200,
    device=device,
    lr=0.01,
    max_iters=5000,
    tol=1e-5
)

print(f"\nFound {len(fixed_points_ctrnn)} fixed points for CT-RNN")
print(f"Residual norms: {residuals_ctrnn}")

In [ ]:
# Analyze stability
print("\nAnalyzing fixed point stability...\n")

stable_fps = []
for i, fp in enumerate(fixed_points_ctrnn):
    # Compute Jacobian
    fp_tensor = torch.tensor(fp, dtype=torch.float32, device=device, requires_grad=True)
    
    # Get dynamics at this point
    dhdt = dynamics_fn_ctrnn(fp_tensor)
    
    # Compute Jacobian using autograd
    jac = torch.autograd.functional.jacobian(dynamics_fn_ctrnn, fp_tensor)
    
    # Analyze stability
    analysis = analyze_fixed_point_stability(jac.detach().cpu().numpy())
    
    if analysis['stable_continuous']:
        stable_fps.append(fp)
        
    print(f"FP {i}: {analysis['classification']}, stable={analysis['stable_continuous']}, max eig={analysis['max_eigenvalue_real']:.3f}")

print(f"\n✓ Found {len(stable_fps)} stable fixed points")
print(f"   Expected: 8 (one per flip-flop state)")

### Visualize Fixed Points in PCA Space

In [ ]:
# Project fixed points to PCA space
if len(fixed_points_ctrnn) > 0:
    fps_array = np.array(fixed_points_ctrnn)
    fps_pca = pca_ctrnn.transform(fps_array)
    
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot hidden states
    ax.scatter(
        hidden_ctrnn_pca[:3000, 0],
        hidden_ctrnn_pca[:3000, 1],
        hidden_ctrnn_pca[:3000, 2],
        c=state_labels[:3000],
        cmap='tab10',
        s=3,
        alpha=0.3,
        label='Hidden states'
    )
    
    # Plot fixed points
    ax.scatter(
        fps_pca[:, 0],
        fps_pca[:, 1],
        fps_pca[:, 2],
        c='red',
        s=200,
        marker='*',
        edgecolors='black',
        linewidths=2,
        label='Fixed points',
        zorder=10
    )
    
    ax.set_xlabel('PC1', fontsize=12)
    ax.set_ylabel('PC2', fontsize=12)
    ax.set_zlabel('PC3', fontsize=12)
    ax.set_title('Fixed Points in State Space', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11, loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📌 Red stars show fixed points - attractors for each flip-flop state!")
else:
    print("No fixed points found to visualize.")

---
## 9. Summary and Comparison

Let's compare the two architectures on the flip-flop task.

In [ ]:
import pandas as pd

# Create comparison table
comparison_data = {
    'Metric': [
        'Parameters',
        'Test MSE Loss',
        'Bit Accuracy (%)',
        'Best Val Loss',
        'PCA Variance (3 PCs)',
        'Fixed Points Found',
        'Time Constant τ'
    ],
    'CT-RNN': [
        f"{n_params:,}",
        f"{test_loss_ctrnn:.6f}",
        f"{accuracy_ctrnn*100:.2f}%",
        f"{min(history_ctrnn['val_loss']):.6f}",
        f"{pca_ctrnn.explained_variance_ratio_[:3].sum():.3f}",
        f"{len(fixed_points_ctrnn)} ({len(stable_fps)} stable)",
        "5.0"
    ],
    'Balanced Rate': [
        f"{n_params_balanced:,}",
        f"{test_loss_balanced:.6f}",
        f"{accuracy_balanced*100:.2f}%",
        f"{min(history_balanced['val_loss']):.6f}",
        f"{pca_balanced.explained_variance_ratio_[:3].sum():.3f}",
        "N/A",
        "τ_E=5.0, τ_I=2.5"
    ]
}

df_comparison = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("ARCHITECTURE COMPARISON ON FLIP-FLOP TASK")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

## Key Insights

### Working Memory Representation
- Both networks successfully learn to maintain flip-flop states over time
- The 8 distinct states (000 to 111) are visible as clusters in PCA space
- Fixed points correspond to stable memory states

### Architecture Trade-offs
- **CT-RNN**: Continuous dynamics, smooth state transitions, explicit time constant
- **Balanced Rate**: E/I structure, biologically-inspired, separate excitatory/inhibitory populations

### Task Characteristics
- **Working memory**: Networks must maintain state with sparse inputs
- **Discrete states**: 8 stable attractors (vs. continuous Lorenz attractor)
- **Longer time constants**: τ=5.0 (vs. τ=1.0 for Lorenz) for memory retention

---

## Next Steps

This notebook demonstrates how to:
1. ✅ Use the framework with a different task (flip-flop vs. Lorenz)
2. ✅ Generate task-specific data in `src/data/`
3. ✅ Train and compare multiple architectures
4. ✅ Visualize state-space representations with PCA
5. ✅ Analyze fixed point structure

**Try your own tasks!** Follow this pattern:
- Create data generation in `src/data/your_task.py`
- Use existing models from `src/models/`
- Apply analysis tools from `src/analysis/`
- Visualize with `src/utils/` plotting functions